In [1]:
import requests 
import json
import sys
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import pytesseract
import numpy as np
import pandas as pd
import datetime
from PIL import Image
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'

In [118]:
def doubleDigit(num):
    if num < 10 :
        return '0'+str(num)
    else:
        return str(num)
    
videoId = '571178493'
clientId = 'vhggq5l1jefc8dtfza2phfc6ih69gs'

chat = []
time = []
user = []

nextCursor = ''

params = {}
params['client_id'] = clientId

i = 0
while True :
        
    if i == 0 :
        URL = 'https://api.twitch.tv/v5/videos/'+videoId+'/comments?content_offset_seconds=0' 
        i += 1
    else:
        URL = 'https://api.twitch.tv/v5/videos/'+videoId+'/comments?cursor=' 
        URL +=  nextCursor   


    response = requests.get(URL, params=params)

    j = json.loads(response.text)

    for k in range(0,len(j["comments"])):
        timer = j["comments"][k]["content_offset_seconds"]

        timeMinute = int(timer/60)

        if timeMinute >= 60 :
            timeHour = int(timeMinute/60)
            timeMinute %= 60
        else :
            timeHour = int(timeMinute/60)

        timeSec = int(timer%60)

        time.append(doubleDigit(timeHour)+':'+doubleDigit(timeMinute)+':'+doubleDigit(timeSec))
        user.append(j["comments"][k]["commenter"]["display_name"])
        chat.append(j["comments"][k]["message"]["body"])
    if '_next' not in j:
        break

    nextCursor = j["_next"]


f = open(videoId+".txt", 'wt', encoding="utf-8")

for x in range(0, len(time)): 
        f.write('[')
        f.write(str(time[x]))
        f.write(']')
        f.write(' ')
        f.write('<')
        f.write(str(user[x]))
        f.write('>')
        f.write(' ')
        f.write(str(chat[x]))
        f.write("\n")

f.close()

In [119]:
time

['00:00:04',
 '00:00:05',
 '00:00:07',
 '00:00:07',
 '00:00:08',
 '00:00:09',
 '00:00:12',
 '00:00:19',
 '00:00:19',
 '00:00:21',
 '00:00:34',
 '00:00:34',
 '00:00:39',
 '00:00:41',
 '00:00:42',
 '00:00:45',
 '00:00:47',
 '00:00:47',
 '00:00:48',
 '00:00:49',
 '00:00:51',
 '00:00:52',
 '00:00:55',
 '00:00:56',
 '00:00:58',
 '00:01:01',
 '00:01:02',
 '00:01:02',
 '00:01:03',
 '00:01:06',
 '00:01:06',
 '00:01:07',
 '00:01:08',
 '00:01:08',
 '00:01:09',
 '00:01:11',
 '00:01:12',
 '00:01:12',
 '00:01:13',
 '00:01:14',
 '00:01:14',
 '00:01:14',
 '00:01:15',
 '00:01:15',
 '00:01:16',
 '00:01:16',
 '00:01:16',
 '00:01:18',
 '00:01:19',
 '00:01:19',
 '00:01:20',
 '00:01:20',
 '00:01:20',
 '00:01:21',
 '00:01:21',
 '00:01:22',
 '00:01:22',
 '00:01:22',
 '00:01:22',
 '00:01:23',
 '00:01:24',
 '00:01:26',
 '00:01:26',
 '00:01:26',
 '00:01:26',
 '00:01:28',
 '00:01:28',
 '00:01:28',
 '00:01:29',
 '00:01:29',
 '00:01:30',
 '00:01:30',
 '00:01:30',
 '00:01:31',
 '00:01:32',
 '00:01:33',
 '00:01:33',

In [120]:
timedata = time[:]

In [5]:
str(datetime.timedelta(milliseconds=3157000))

'0:52:37'

In [6]:
str(datetime.timedelta(seconds=666))

'0:11:06'

In [7]:
def time_msec(time):
    time_split=time.split(':')
    if len(time_split) == 2 :
        sec = int(time_split[0])*60+int(time_split[1])
        return sec*1000
    elif len(time_split) == 3 :
        sec = (int(time_split[0])*60+int(time_split[1]))*60+int(time_split[2])
        return sec*1000

def msec_time(msec):
    sec = msec/1000
    return str(int(sec//60))+':'+str(int(sec%60))

In [9]:
def ocr(frame):
    data=[]

    board=frame[:25,1550:1905]
    length = board.shape[1]
    board=board[:,5:int(length*0.21)]

    sharpening_1 = np.array([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]])
    board = cv2.filter2D(board, -1, sharpening_1)

    fx=4
    fy=4
    board=cv2.resize(board, dsize=(0,0), fx=fx,fy=fy)


    board = cv2.cvtColor(board, cv2.COLOR_RGB2GRAY)

    kernel = np.ones((3,3),np.uint8)


    board = cv2.GaussianBlur(board, (3, 3), 0)
    _, board = cv2.threshold(board, 0, 255,  cv2.THRESH_OTSU+cv2.THRESH_BINARY_INV)

#     print(pytesseract.image_to_string(board))
    return pytesseract.image_to_string(board)

In [10]:
def ocr2(frame):
    data=[]
    board=frame[:25,1550:1905]
    gray = cv2.cvtColor(board, cv2.COLOR_RGB2GRAY)

    fx=4
    fy=4

    board=cv2.resize(gray, dsize=(0,0), fx=fx,fy=fy)

    img_blur = cv2.GaussianBlur(board, (3,3), 0)
    _, board = cv2.threshold(img_blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    length = board.shape[1]

#     score = board[:,5:int(length*0.21)]

#     str_score = pytesseract.image_to_string(score)
#     print(str_score)
#     data.append(str_score)

    kda = board[:,int(length*0.32):int(length*0.48)]

    str_kda = pytesseract.image_to_string(kda)
#     print(str_kda)
    data.append(str_kda)
    
    time = board[:,int(length*0.87):]
    str_time = pytesseract.image_to_string(time)
    data.append(str_time)

    return data

In [275]:
def ocr3(frame):
#     img=cv2.imread('video/image ('+str(i)+').jpg')
    
    board=frame[1047:1070,1180:1237]
    gray = cv2.cvtColor(board, cv2.COLOR_RGB2GRAY)
    
    fx=4
    fy=4
    board=cv2.resize(gray, dsize=(0,0), fx=fx,fy=fy)
    board = cv2.GaussianBlur(board, (3, 3), 0)
    _, board = cv2.threshold(board, 0, 255,  cv2.THRESH_OTSU+cv2.THRESH_BINARY_INV)

    # plt.imshow(board)

    # img[1043:1070,1180:1230]
    str_time = pytesseract.image_to_string(board)
    return str_time

In [279]:
# 동영상 프로세싱
import cv2

cap = cv2.VideoCapture('video/full.mp4')

starttime = '38:34'
endtime = '1:15:15'
cap.set(cv2.CAP_PROP_POS_MSEC, time_msec(starttime))

i=0
datalist=[]
while (cap.isOpened):
    i += 1
    ret, frame = cap.read()
    if ret:
        if i%60==0:
#             dt=ocr(frame)
#             dt2=ocr2(frame)
#             data = [dt]+dt2+[cap.get(cv2.CAP_PROP_POS_MSEC)]
            dt = ocr3(frame)
            data = [dt]+[cap.get(cv2.CAP_PROP_POS_MSEC)]
            if i%600==0:
                print(data)
                
            datalist.append(data)
#             print(cap.get(cv2.CAP_PROP_POS_MSEC)) # 동영상 재생시간
        if cap.get(cv2.CAP_PROP_POS_MSEC) > time_msec(endtime):
            break
    else:
        break

['', 2323984.0]
['', 2333984.0]
['', 2343984.0]
['', 2353984.0]
['', 2363984.0]
['', 2373984.0]
['', 2383984.0]
['21', 2393984.0]
['63', 2403984.0]
['97', 2413984.0]
['152', 2423984.0]
['194', 2433984.0]
['228', 2443984.0]
['298', 2453984.0]
['360', 2463984.0]
['440', 2473984.0]
['524', 2483984.0]
['579', 2493984.0]
['635', 2503984.0]
['683', 2513984.0]
['738', 2523984.0]
['787', 2533984.0]
['8/0', 2543984.0]
['996', 2553984.0]
['716', 2563984.0]
['36', 2573984.0]
['92', 2583984.0]
['112', 2593984.0]
['', 2603984.0]
['', 2613984.0]
['98', 2623984.0]
['119', 2633984.0]
['139', 2643984.0]
['160', 2653984.0]
['264', 2663984.0]
['284', 2673984.0]
['305', 2683984.0]
['325', 2693984.0]
['416', 2703984.0]
['471', 2713984.0]
['491', 2723984.0]
['554', 2733984.0]
['706', 2743984.0]
['832', 2753984.0]
['852', 2763984.0]
['172', 2773984.0]
['', 2783984.0]
['213', 2793984.0]
['234', 2803984.0]
['254', 2813984.0]
['744', 2823984.0]
['921', 2833984.0]
['941', 2843984.0]
['962', 2853984.0]
['1087', 2

In [280]:
datalist

[['', 2314984.0],
 ['', 2315984.0],
 ['', 2316984.0],
 ['', 2317984.0],
 ['', 2318984.0],
 ['', 2319984.0],
 ['', 2320984.0],
 ['', 2321984.0],
 ['', 2322984.0],
 ['', 2323984.0],
 ['', 2324984.0],
 ['', 2325984.0],
 ['', 2326984.0],
 ['', 2327984.0],
 ['', 2328984.0],
 ['', 2329984.0],
 ['', 2330984.0],
 ['', 2331984.0],
 ['', 2332984.0],
 ['', 2333984.0],
 ['', 2334984.0],
 ['', 2335984.0],
 ['', 2336984.0],
 ['', 2337984.0],
 ['', 2338984.0],
 ['', 2339984.0],
 ['', 2340984.0],
 ['', 2341984.0],
 ['', 2342984.0],
 ['', 2343984.0],
 ['', 2344984.0],
 ['', 2345984.0],
 ['', 2346984.0],
 ['', 2347984.0],
 ['', 2348984.0],
 ['', 2349984.0],
 ['', 2350984.0],
 ['', 2351984.0],
 ['', 2352984.0],
 ['', 2353984.0],
 ['', 2354984.0],
 ['', 2355984.0],
 ['', 2356984.0],
 ['', 2357984.0],
 ['', 2358984.0],
 ['', 2359984.0],
 ['', 2360984.0],
 ['', 2361984.0],
 ['', 2362984.0],
 ['', 2363984.0],
 ['', 2364984.0],
 ['', 2365984.0],
 ['', 2366984.0],
 ['', 2367984.0],
 ['', 2368984.0],
 ['', 2369

In [189]:
df=pd.DataFrame(datalist)
df.head()

,0,1,2,3
0,0 vsQ,0/0/0,00:20,2314984.0
1,,0/0/0,00:21,2315984.0
2,0 vs 0,0/0/0,00:22,2316984.0
3,0 vsO,0/0/0,00:23,2317984.0
4,Us _,0/0/0,00:24,2318984.0


In [190]:
# kda 제대로 출력되었는지 확인. 세 값이 모두 있지 않을경우 Na처리.
for i in range(len(df)):
    if len(df[1][i].split('/'))!=3:
        df[1][i]=np.nan

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [191]:
# score를 vs 기준으로 나누고, kda를 / 기준으로 나눠서 새로운 col 생성
df['fr']=df[0].str.split('vs').str[0]
df['en']=df[0].str.split('vs').str[1]
df['k']=df[1].str.split('/').str[0]
df['d']=df[1].str.split('/').str[1]
df['a']=df[1].str.split('/').str[2]

In [192]:
df

,0,1,2,3,fr,en,k,d,a
0,0 vsQ,0/0/0,00:20,2314984.0,0,Q,0,0,0
1,,0/0/0,00:21,2315984.0,,NaN,0,0,0
2,0 vs 0,0/0/0,00:22,2316984.0,0,0,0,0,0
3,0 vsO,0/0/0,00:23,2317984.0,0,O,0,0,0
4,Us _,0/0/0,00:24,2318984.0,Us _,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...
2196,,14/5/17,36:56,4510984.0,,NaN,14,5,17
2197,,14/5/17,36:57,4511984.0,,NaN,14,5,17
2198,39 vs AZ,14/5/1),36:58,4512984.0,39,AZ,14,5,1)
2199,39 vs AZ,14/5/1),36:59,4513984.0,39,AZ,14,5,1)


In [193]:
# object 타입을 숫자로 변환. 숫자가 아닐시 강제로(Na)
df['fr']=pd.to_numeric(df['fr'], errors='coerce')
df['en']=pd.to_numeric(df['en'], errors='coerce')
df['k']=pd.to_numeric(df['k'], errors='coerce')
df['d']=pd.to_numeric(df['d'], errors='coerce')
df['a']=pd.to_numeric(df['a'], errors='coerce')
df

,0,1,2,3,fr,en,k,d,a
0,0 vsQ,0/0/0,00:20,2314984.0,0.0,NaN,0.0,0.0,0.0
1,,0/0/0,00:21,2315984.0,NaN,NaN,0.0,0.0,0.0
2,0 vs 0,0/0/0,00:22,2316984.0,0.0,0.0,0.0,0.0,0.0
3,0 vsO,0/0/0,00:23,2317984.0,0.0,NaN,0.0,0.0,0.0
4,Us _,0/0/0,00:24,2318984.0,NaN,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2196,,14/5/17,36:56,4510984.0,NaN,NaN,14.0,5.0,17.0
2197,,14/5/17,36:57,4511984.0,NaN,NaN,14.0,5.0,17.0
2198,39 vs AZ,14/5/1),36:58,4512984.0,39.0,NaN,14.0,5.0,NaN
2199,39 vs AZ,14/5/1),36:59,4513984.0,39.0,NaN,14.0,5.0,NaN


In [137]:
# ffill을 통해 nan값 채워주기
# df['k'].fillna(method='bfill', inplace=True)
# df['d'].fillna(method='bfill', inplace=True)
# df['a'].fillna(method='bfill', inplace=True)
# df['fr'].fillna(method='bfill', inplace=True)
# df['en'].fillna(method='bfill', inplace=True)
# df

,0,1,2,3,fr,en,k,d,a
0,0 vsQ,0/0/0,00:20,2314984.0,0.0,0.0,0.0,0.0,0.0
1,,0/0/0,00:21,2315984.0,0.0,0.0,0.0,0.0,0.0
2,0 vs 0,0/0/0,00:22,2316984.0,0.0,0.0,0.0,0.0,0.0
3,0 vsO,0/0/0,00:23,2317984.0,0.0,0.0,0.0,0.0,0.0
4,Us _,0/0/0,00:24,2318984.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2196,,14/5/17,36:56,4510984.0,39.0,NaN,14.0,5.0,17.0
2197,,14/5/17,36:57,4511984.0,39.0,NaN,14.0,5.0,17.0
2198,39 vs AZ,14/5/1),36:58,4512984.0,39.0,NaN,14.0,5.0,17.0
2199,39 vs AZ,14/5/1),36:59,4513984.0,39.0,NaN,14.0,5.0,17.0


In [138]:
# df['k'].fillna(method='ffill', inplace=True)
# df['d'].fillna(method='ffill', inplace=True)
# df['a'].fillna(method='ffill', inplace=True)
# df['fr'].fillna(method='ffill', inplace=True)
# df['en'].fillna(method='ffill', inplace=True)
# df

,0,1,2,3,fr,en,k,d,a
0,0 vsQ,0/0/0,00:20,2314984.0,0.0,0.0,0.0,0.0,0.0
1,,0/0/0,00:21,2315984.0,0.0,0.0,0.0,0.0,0.0
2,0 vs 0,0/0/0,00:22,2316984.0,0.0,0.0,0.0,0.0,0.0
3,0 vsO,0/0/0,00:23,2317984.0,0.0,0.0,0.0,0.0,0.0
4,Us _,0/0/0,00:24,2318984.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2196,,14/5/17,36:56,4510984.0,39.0,41.0,14.0,5.0,17.0
2197,,14/5/17,36:57,4511984.0,39.0,41.0,14.0,5.0,17.0
2198,39 vs AZ,14/5/1),36:58,4512984.0,39.0,41.0,14.0,5.0,17.0
2199,39 vs AZ,14/5/1),36:59,4513984.0,39.0,41.0,14.0,5.0,17.0


In [194]:
df['time']=df[3].map(lambda x : str(datetime.timedelta(milliseconds=x-984)))
df.head()

,0,1,2,3,fr,en,k,d,a,time
0,0 vsQ,0/0/0,00:20,2314984.0,0.0,NaN,0.0,0.0,0.0,0:38:34
1,,0/0/0,00:21,2315984.0,NaN,NaN,0.0,0.0,0.0,0:38:35
2,0 vs 0,0/0/0,00:22,2316984.0,0.0,0.0,0.0,0.0,0.0,0:38:36
3,0 vsO,0/0/0,00:23,2317984.0,0.0,NaN,0.0,0.0,0.0,0:38:37
4,Us _,0/0/0,00:24,2318984.0,NaN,NaN,0.0,0.0,0.0,0:38:38


In [195]:
df.set_index('time', inplace=True)
df.head()

,0,1,2,3,fr,en,k,d,a
time,,,,,,,,,
0:38:34,0 vsQ,0/0/0,00:20,2314984.0,0.0,NaN,0.0,0.0,0.0
0:38:35,,0/0/0,00:21,2315984.0,NaN,NaN,0.0,0.0,0.0
0:38:36,0 vs 0,0/0/0,00:22,2316984.0,0.0,0.0,0.0,0.0,0.0
0:38:37,0 vsO,0/0/0,00:23,2317984.0,0.0,NaN,0.0,0.0,0.0
0:38:38,Us _,0/0/0,00:24,2318984.0,NaN,NaN,0.0,0.0,0.0


In [141]:
# for i in range(len(timedata)) :
#     timedata[i] = timedata[i][1:]

In [142]:
timedata

['0:00:04',
 '0:00:05',
 '0:00:07',
 '0:00:07',
 '0:00:08',
 '0:00:09',
 '0:00:12',
 '0:00:19',
 '0:00:19',
 '0:00:21',
 '0:00:34',
 '0:00:34',
 '0:00:39',
 '0:00:41',
 '0:00:42',
 '0:00:45',
 '0:00:47',
 '0:00:47',
 '0:00:48',
 '0:00:49',
 '0:00:51',
 '0:00:52',
 '0:00:55',
 '0:00:56',
 '0:00:58',
 '0:01:01',
 '0:01:02',
 '0:01:02',
 '0:01:03',
 '0:01:06',
 '0:01:06',
 '0:01:07',
 '0:01:08',
 '0:01:08',
 '0:01:09',
 '0:01:11',
 '0:01:12',
 '0:01:12',
 '0:01:13',
 '0:01:14',
 '0:01:14',
 '0:01:14',
 '0:01:15',
 '0:01:15',
 '0:01:16',
 '0:01:16',
 '0:01:16',
 '0:01:18',
 '0:01:19',
 '0:01:19',
 '0:01:20',
 '0:01:20',
 '0:01:20',
 '0:01:21',
 '0:01:21',
 '0:01:22',
 '0:01:22',
 '0:01:22',
 '0:01:22',
 '0:01:23',
 '0:01:24',
 '0:01:26',
 '0:01:26',
 '0:01:26',
 '0:01:26',
 '0:01:28',
 '0:01:28',
 '0:01:28',
 '0:01:29',
 '0:01:29',
 '0:01:30',
 '0:01:30',
 '0:01:30',
 '0:01:31',
 '0:01:32',
 '0:01:33',
 '0:01:33',
 '0:01:33',
 '0:01:33',
 '0:01:34',
 '0:01:34',
 '0:01:37',
 '0:01:37',
 '0:

In [196]:
timeseries = pd.Series(timedata)

In [197]:
timeseries

0        0:00:04
1        0:00:05
2        0:00:07
3        0:00:07
4        0:00:08
          ...   
26391    2:24:01
26392    2:24:01
26393    2:24:01
26394    2:24:02
26395    2:24:02
Length: 26396, dtype: object

In [198]:
df2 = pd.DataFrame(index=timeseries.unique())

In [199]:
timeseries.value_counts()

2:00:02    15
1:26:21    15
1:44:30    15
2:09:52    15
2:11:14    15
           ..
1:05:50     1
1:06:54     1
1:05:32     1
1:57:14     1
1:18:56     1
Length: 7668, dtype: int64

In [200]:
df2['count'] = timeseries.value_counts()

In [201]:
df2.index.name = 'time'

In [202]:
df2.head()

,count
time,
0:00:04,1
0:00:05,1
0:00:07,2
0:00:08,1
0:00:09,1


In [203]:
df3 = pd.merge(df,df2,on='time')

In [204]:
df3.head()

,0,1,2,3,fr,en,k,d,a,count
time,,,,,,,,,,
0:38:35,,0/0/0,00:21,2315984.0,NaN,NaN,0.0,0.0,0.0,1
0:38:36,0 vs 0,0/0/0,00:22,2316984.0,0.0,0.0,0.0,0.0,0.0,4
0:38:37,0 vsO,0/0/0,00:23,2317984.0,0.0,NaN,0.0,0.0,0.0,2
0:38:39,0 vsO,0/0/0,00:25,2319984.0,0.0,NaN,0.0,0.0,0.0,6
0:38:40,OvsQ,0/0/0,00:26,2320984.0,NaN,NaN,0.0,0.0,0.0,6


In [205]:
df3.tail()

,0,1,2,3,fr,en,k,d,a,count
time,,,,,,,,,,
1:15:10,,14/5/17,36:56,4510984.0,NaN,NaN,14.0,5.0,17.0,2
1:15:11,,14/5/17,36:57,4511984.0,NaN,NaN,14.0,5.0,17.0,4
1:15:12,39 vs AZ,14/5/1),36:58,4512984.0,39.0,NaN,14.0,5.0,NaN,5
1:15:13,39 vs AZ,14/5/1),36:59,4513984.0,39.0,NaN,14.0,5.0,NaN,3
1:15:14,39 vs A?,14/5/17,37:00,4514984.0,39.0,NaN,14.0,5.0,17.0,3


In [206]:
df3.describe()

,3,fr,en,k,d,a,count
count,1.954000e+03,1337.000000,1457.000000,1951.000000,1948.000000,1936.000000,1954.000000
mean,3.413244e+06,23.376664,19.064434,5.485905,2.929671,2.889979,3.307574
std,6.367273e+05,24.482905,11.051447,4.470264,1.938363,3.364626,2.502855
min,2.315984e+06,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2.860234e+06,15.000000,10.000000,1.000000,1.000000,0.000000,2.000000
50%,3.405484e+06,22.000000,19.000000,8.000000,3.000000,1.000000,3.000000
75%,3.964484e+06,33.000000,25.000000,8.000000,5.000000,5.000000,4.000000
max,4.514984e+06,715.000000,41.000000,14.000000,5.000000,17.000000,15.000000


In [207]:
print(df3['fr'].unique())
print(df3['en'].unique())
print(df3['k'].unique())
print(df3['d'].unique())
print(df3['a'].unique())

[  nan   0.    1.    2.    3.    4.    5.    6.    7.    8.    9.   10.
 110.   11.   17.   14.   12.   13.   15.  715.   16.  146.    1.6  18.
  19.   20.   21.   22.   24.   25.   26.   27.   28.   29.   30.   34.
 371.   31.   37.   32.   33.   35.   36.   46.   39. ]
[  nan  0.    1.    2.    3.    4.    5.    6.    7.    9.   10.   11.
 12.   13.   14.   15.   16.   17.   18.    0.18 19.   20.   21.   22.
 23.   24.   25.   26.   27.   28.   29.   30.   31.   32.   33.    0.33
 34.   35.   36.   37.    0.37 39.   40.   41.  ]
[ 0.  1.  2.  3. nan  4.  9.  6.  7.  8. 10. 11. 12. 13. 14.]
[ 0.  1. nan  2.  3.  4.  5.]
[ 0.  1. 11. nan  2.  3.  4.  5.  6.  7.  8.  9. 17.]


In [208]:
print(df3['fr'].unique())
print(df3['en'].unique())

[  nan   0.    1.    2.    3.    4.    5.    6.    7.    8.    9.   10.
 110.   11.   17.   14.   12.   13.   15.  715.   16.  146.    1.6  18.
  19.   20.   21.   22.   24.   25.   26.   27.   28.   29.   30.   34.
 371.   31.   37.   32.   33.   35.   36.   46.   39. ]
[  nan  0.    1.    2.    3.    4.    5.    6.    7.    9.   10.   11.
 12.   13.   14.   15.   16.   17.   18.    0.18 19.   20.   21.   22.
 23.   24.   25.   26.   27.   28.   29.   30.   31.   32.   33.    0.33
 34.   35.   36.   37.    0.37 39.   40.   41.  ]


In [211]:
df3.to_excel('df3.xlsx')

In [209]:
df3['kda'] = df3['k']+df3['d']+df3['a']

In [210]:
df3

,0,1,2,3,fr,en,k,d,a,count,kda
time,,,,,,,,,,,
0:38:35,,0/0/0,00:21,2315984.0,NaN,NaN,0.0,0.0,0.0,1,0.0
0:38:36,0 vs 0,0/0/0,00:22,2316984.0,0.0,0.0,0.0,0.0,0.0,4,0.0
0:38:37,0 vsO,0/0/0,00:23,2317984.0,0.0,NaN,0.0,0.0,0.0,2,0.0
0:38:39,0 vsO,0/0/0,00:25,2319984.0,0.0,NaN,0.0,0.0,0.0,6,0.0
0:38:40,OvsQ,0/0/0,00:26,2320984.0,NaN,NaN,0.0,0.0,0.0,6,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1:15:10,,14/5/17,36:56,4510984.0,NaN,NaN,14.0,5.0,17.0,2,36.0
1:15:11,,14/5/17,36:57,4511984.0,NaN,NaN,14.0,5.0,17.0,4,36.0
1:15:12,39 vs AZ,14/5/1),36:58,4512984.0,39.0,NaN,14.0,5.0,NaN,5,NaN


In [212]:
exdf = df3.copy()

In [250]:
exdf

,0,1,2,3,fr,en,k,d,a,count,kda,d_kda
time,,,,,,,,,,,,
0:38:35,,0/0/0,00:21,2315984.0,NaN,NaN,0.0,0.0,0.0,1,0.0,0.0
0:38:36,0 vs 0,0/0/0,00:22,2316984.0,0.0,0.0,0.0,0.0,0.0,4,0.0,NaN
0:38:37,0 vsO,0/0/0,00:23,2317984.0,0.0,NaN,0.0,0.0,0.0,2,0.0,0.0
0:38:39,0 vsO,0/0/0,00:25,2319984.0,0.0,NaN,0.0,0.0,0.0,6,0.0,0.0
0:38:40,OvsQ,0/0/0,00:26,2320984.0,NaN,NaN,0.0,0.0,0.0,6,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1:15:10,,14/5/17,36:56,4510984.0,NaN,NaN,14.0,5.0,17.0,2,36.0,0.0
1:15:11,,14/5/17,36:57,4511984.0,NaN,NaN,14.0,5.0,17.0,4,36.0,0.0
1:15:12,39 vs AZ,14/5/1),36:58,4512984.0,39.0,NaN,14.0,5.0,NaN,5,NaN,NaN


In [217]:
exdf['d_kda']=pd.Series()

In [219]:
for i in reversed(range(len(exdf))):
#     print(i)
    exdf['d_kda'][i] = exdf['kda'][i]-exdf['kda'][i-15]

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [188]:
print(exdf['fr'].unique())
print(exdf['en'].unique())
print(exdf['k'].unique())
print(exdf['d'].unique())
print(exdf['a'].unique())

[  0.    1.    2.    3.    4.    5.    6.    7.    8.    9.   10.  110.
  11.   17.   14.   12.   13.   15.  715.   16.  146.    1.6  18.   19.
  20.   21.   22.   24.   25.   26.   27.   23.   28.   29.   30.   34.
 371.   31.   37.   32.   33.   35.   36.   46.   39. ]
[ 0.    1.    2.    3.    4.    5.    6.    7.    9.   10.   11.   12.
 13.   14.   15.   16.   17.   18.    0.18 19.   20.   21.   22.   23.
 24.   25.   26.   27.   28.   29.   30.   31.   32.   33.    0.33 34.
 35.   36.   37.    0.37 39.   40.   41.  ]
[ 0.  1.  2.  3.  4.  9.  6.  7.  8. 10. 11. 12. 13. 14.]
[0. 1. 2. 3. 4. 5.]
[ 0.  1. 11.  2.  3.  4.  5.  6.  7.  8.  9. 17.]


In [247]:
exdf['d_kda'][exdf['d_kda']>5] = 0
exdf['d_kda'][exdf['d_kda']<0] = 0

In [251]:
exdf['d_k']=pd.Series()
for i in reversed(range(len(exdf))):
#     print(i)
    exdf['d_k'][i] = exdf['k'][i]-exdf['k'][i-15]

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [253]:
exdf['d_d']=pd.Series()
for i in reversed(range(len(exdf))):
#     print(i)
    exdf['d_d'][i] = exdf['d'][i]-exdf['d'][i-15]

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [254]:
exdf['d_a']=pd.Series()
for i in reversed(range(len(exdf))):
#     print(i)
    exdf['d_a'][i] = exdf['a'][i]-exdf['a'][i-15]

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [256]:
exdf['d_k'][exdf['d_k']>5] = 0
exdf['d_k'][exdf['d_k']<0] = 0
exdf['d_d'][exdf['d_d']>5] = 0
exdf['d_d'][exdf['d_d']<0] = 0
exdf['d_a'][exdf['d_a']>5] = 0
exdf['d_a'][exdf['d_a']<0] = 0

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Anac

In [257]:
exdf

,0,1,2,3,fr,en,k,d,a,count,kda,d_kda,d_k,d_d,d_a
time,,,,,,,,,,,,,,,
0:38:35,,0/0/0,00:21,2315984.0,NaN,NaN,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0
0:38:36,0 vs 0,0/0/0,00:22,2316984.0,0.0,0.0,0.0,0.0,0.0,4,0.0,NaN,0.0,0.0,NaN
0:38:37,0 vsO,0/0/0,00:23,2317984.0,0.0,NaN,0.0,0.0,0.0,2,0.0,0.0,0.0,0.0,0.0
0:38:39,0 vsO,0/0/0,00:25,2319984.0,0.0,NaN,0.0,0.0,0.0,6,0.0,0.0,0.0,0.0,0.0
0:38:40,OvsQ,0/0/0,00:26,2320984.0,NaN,NaN,0.0,0.0,0.0,6,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1:15:10,,14/5/17,36:56,4510984.0,NaN,NaN,14.0,5.0,17.0,2,36.0,0.0,0.0,0.0,0.0
1:15:11,,14/5/17,36:57,4511984.0,NaN,NaN,14.0,5.0,17.0,4,36.0,0.0,0.0,0.0,0.0
1:15:12,39 vs AZ,14/5/1),36:58,4512984.0,39.0,NaN,14.0,5.0,NaN,5,NaN,NaN,0.0,0.0,NaN


In [260]:
exdf['d_k'].fillna(method='ffill', inplace=True)
exdf['d_d'].fillna(method='ffill', inplace=True)
exdf['d_a'].fillna(method='ffill', inplace=True)

In [262]:
exdf['n_chat']=pd.Series()

In [264]:
sum(exdf['count'][1:5])

18

In [268]:
for i in range(len(exdf)):
    exdf['n_chat'][i] = sum(exdf['count'][i:i+15])

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [269]:
exdf

,0,1,2,3,fr,en,k,d,a,count,kda,d_kda,d_k,d_d,d_a,n_chat
time,,,,,,,,,,,,,,,,
0:38:35,,0/0/0,00:21,2315984.0,NaN,NaN,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,67
0:38:36,0 vs 0,0/0/0,00:22,2316984.0,0.0,0.0,0.0,0.0,0.0,4,0.0,NaN,0.0,0.0,0.0,69
0:38:37,0 vsO,0/0/0,00:23,2317984.0,0.0,NaN,0.0,0.0,0.0,2,0.0,0.0,0.0,0.0,0.0,66
0:38:39,0 vsO,0/0/0,00:25,2319984.0,0.0,NaN,0.0,0.0,0.0,6,0.0,0.0,0.0,0.0,0.0,67
0:38:40,OvsQ,0/0/0,00:26,2320984.0,NaN,NaN,0.0,0.0,0.0,6,0.0,0.0,0.0,0.0,0.0,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1:15:10,,14/5/17,36:56,4510984.0,NaN,NaN,14.0,5.0,17.0,2,36.0,0.0,0.0,0.0,0.0,17
1:15:11,,14/5/17,36:57,4511984.0,NaN,NaN,14.0,5.0,17.0,4,36.0,0.0,0.0,0.0,0.0,15
1:15:12,39 vs AZ,14/5/1),36:58,4512984.0,39.0,NaN,14.0,5.0,NaN,5,NaN,NaN,0.0,0.0,0.0,11


In [283]:
gold=pd.DataFrame(datalist)

In [288]:
gold

,0,1
time,,
0:38:34,,2314984.0
0:38:35,,2315984.0
0:38:36,,2316984.0
0:38:37,,2317984.0
0:38:38,,2318984.0
...,...,...
1:15:10,1811,4510984.0
1:15:11,1833,4511984.0
1:15:12,1835,4512984.0


In [285]:
gold['time']=gold[1].map(lambda x : str(datetime.timedelta(milliseconds=x-984)))

In [287]:
gold.set_index('time', inplace=True)

In [291]:
gold[0]=pd.to_numeric(gold[0], errors='coerce')

In [292]:
gold

,0,1
time,,
0:38:34,NaN,2314984.0
0:38:35,NaN,2315984.0
0:38:36,NaN,2316984.0
0:38:37,NaN,2317984.0
0:38:38,NaN,2318984.0
...,...,...
1:15:10,1811.0,4510984.0
1:15:11,1833.0,4511984.0
1:15:12,1835.0,4512984.0


In [348]:
gold.to_excel('gold.xlsx')

In [294]:
exdf['gold'] = gold[0]

In [349]:
exdf

,0,1,2,3,fr,en,k,d,a,count,kda,d_kda,d_k,d_d,d_a,n_chat,gold,d_gold
time,,,,,,,,,,,,,,,,,,
0:38:35,,0/0/0,00:21,2315984.0,NaN,NaN,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,67,NaN,NaN
0:38:36,0 vs 0,0/0/0,00:22,2316984.0,0.0,0.0,0.0,0.0,0.0,4,0.0,NaN,0.0,0.0,0.0,69,NaN,NaN
0:38:37,0 vsO,0/0/0,00:23,2317984.0,0.0,NaN,0.0,0.0,0.0,2,0.0,0.0,0.0,0.0,0.0,66,NaN,NaN
0:38:39,0 vsO,0/0/0,00:25,2319984.0,0.0,NaN,0.0,0.0,0.0,6,0.0,0.0,0.0,0.0,0.0,67,NaN,NaN
0:38:40,OvsQ,0/0/0,00:26,2320984.0,NaN,NaN,0.0,0.0,0.0,6,0.0,0.0,0.0,0.0,0.0,65,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1:15:10,,14/5/17,36:56,4510984.0,NaN,NaN,14.0,5.0,17.0,2,36.0,0.0,0.0,0.0,0.0,17,1811.0,284.0
1:15:11,,14/5/17,36:57,4511984.0,NaN,NaN,14.0,5.0,17.0,4,36.0,0.0,0.0,0.0,0.0,15,1833.0,254.0
1:15:12,39 vs AZ,14/5/1),36:58,4512984.0,39.0,NaN,14.0,5.0,NaN,5,NaN,NaN,0.0,0.0,0.0,11,1835.0,200.0


In [297]:
exdf['d_gold'] = pd.Series()

In [347]:
for i in range(len(exdf)):
    exdf['d_gold'][i] = exdf['gold'][i] - exdf['gold'][i-15]

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [351]:
exdf['d_gold'][exdf['d_gold']<0] = np.nan

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [352]:
exdf['d_gold'][exdf['d_gold']>2000] = np.nan

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [350]:
exdf.to_excel('exdf.xlsx')

In [353]:
dataset = exdf[['d_kda','d_k','d_d','d_a','d_gold','n_chat']]

In [379]:
dataset.to_excel('dataset.xlsx')

In [377]:
dataset.describe()

,d_kda,d_k,d_d,d_a,d_gold,n_chat
count,1264.000000,1264.000000,1264.000000,1264.000000,1264.000000,1264.000000
mean,0.282437,0.140032,0.044304,0.101266,0.134941,0.139241
std,0.714165,0.455645,0.205851,0.398967,0.137109,0.346334
min,0.000000,0.000000,0.000000,0.000000,0.021723,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.032772,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.091386,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.169288,0.000000
max,5.000000,5.000000,1.000000,3.000000,1.000000,1.000000


In [378]:
dataset

,d_kda,d_k,d_d,d_a,d_gold,n_chat
time,,,,,,
0:40:08,0.0,0.0,0.0,0.0,0.038951,0
0:40:09,0.0,0.0,0.0,0.0,0.519850,0
0:40:11,0.0,0.0,0.0,0.0,0.053933,0
0:40:14,0.0,0.0,0.0,0.0,0.058427,0
0:40:17,0.0,0.0,0.0,0.0,0.073408,0
...,...,...,...,...,...,...
1:15:08,0.0,0.0,0.0,0.0,0.176779,0
1:15:09,0.0,0.0,0.0,0.0,0.176779,0
1:15:10,0.0,0.0,0.0,0.0,0.212734,0


In [356]:
line = dataset['n_chat'].mean()+dataset['n_chat'].std()
print(line)

74.3221667794736


In [357]:
for i in range(len(dataset)):
    if dataset['n_chat'][i] > line :
        dataset['n_chat'][i] = 1
    else :
        dataset['n_chat'][i] = 0

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [370]:
dataset.dropna(axis=0, inplace=True)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [365]:
dataset.fillna(value=99999)

,d_kda,d_k,d_d,d_a,d_gold,n_chat
time,,,,,,
0:38:35,0.0,0.0,0.0,0.0,99999.0,0
0:38:36,99999.0,0.0,0.0,0.0,99999.0,0
0:38:37,0.0,0.0,0.0,0.0,99999.0,0
0:38:39,0.0,0.0,0.0,0.0,99999.0,0
0:38:40,0.0,0.0,0.0,0.0,99999.0,0
...,...,...,...,...,...,...
1:15:10,0.0,0.0,0.0,0.0,284.0,0
1:15:11,0.0,0.0,0.0,0.0,254.0,0
1:15:12,99999.0,0.0,0.0,0.0,200.0,0


In [373]:
dataset['d_gold'] = dataset['d_gold']/dataset['d_gold'].max()

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
